In [25]:
! curl -L https://git.io/GeoLite2-City.mmdb -o GeoLite2-City.mmdb 
! pip3 install geoip2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   152  100   152    0     0    388      0 --:--:-- --:--:-- --:--:--   388
100   626  100   626    0     0   1282      0 --:--:-- --:--:-- --:--:--  1282
100 68.4M  100 68.4M    0     0  38.8M      0  0:00:01  0:00:01 --:--:-- 60.3M
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [5]:
import requests
import zipfile
import io
import pandas as pd
from nslookup import Nslookup
import warnings
import geoip2.database
import folium
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker
warnings.filterwarnings("ignore")

In [6]:
r = requests.get("http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [7]:
df = pd.read_csv ('top-1m.csv', names=['Rank', 'DNS'], header=None)
df['DNS'] = df['DNS'].astype('str')

In [23]:
dns_query = Nslookup(dns_servers=["8.8.8.8"])
top1k = df.head(10);
top1k['ip_address'] = top1k['DNS'].map(lambda host:dns_query.dns_lookup(host).answer)
top1k["ip_address"] = top1k["ip_address"].str[0]
top1k = top1k.dropna()

In [31]:
def get_latitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.latitude
        return output
    except KeyError:
        return pd.np.nan

def get_longitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.longitude
        return output
    except KeyError:
        return pd.np.nan


def get_country(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.country.iso_code
        return output
    except KeyError:
        return pd.np.nan


unique_ips = top1k['ip_address'].unique()
unique_ips = pd.Series(unique_ips, index = unique_ips)
top1k['Latitude'] = top1k['ip_address'].map(unique_ips.apply(get_latitude))
top1k['Longitude'] = top1k['ip_address'].map(unique_ips.apply(get_longitude))
top1k['Country'] = top1k['ip_address'].map(unique_ips.apply(get_country))



In [32]:
top1k = top1k.dropna()
top1k

,Rank,DNS,ip_address,Latitude,Longitude,Country
0,1,google.com,142.250.72.78,37.751,-97.822,US
1,2,netflix.com,52.3.144.142,39.0469,-77.4903,US
2,3,ftl.netflix.com,45.57.41.1,37.751,-97.822,US
4,5,api-global.netflix.com,52.33.148.94,45.8234,-119.7257,US
5,6,microsoft.com,104.215.148.63,1.3036,103.8554,SG
6,7,www.google.com,142.251.35.196,37.751,-97.822,US
7,8,nrdp.prod.ftl.netflix.com,34.216.127.28,45.8234,-119.7257,US
8,9,ichnaea.netflix.com,34.210.254.90,45.8234,-119.7257,US
9,10,data.microsoft.com,40.112.143.140,37.3388,-121.8916,US


In [34]:
m = folium.Map(location=[50, 0], tiles='cartodb positron', zoom_start=3)

#You Markler the point in Map
for indice, row in top1k.iterrows():
    popup_txt = (row['DNS'] + '\n' + row['ip_address'])
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=popup_txt
    ).add_to(m)


m
